# Merging DataFrames

In [2]:
import pandas as pd

## Our Dataset
- Our datasets are spread across multiple files in this section. Each file has a `restaurant_` prefix.
- The `customers.csv` file stores our restaurant's customers.
- The `foods.csv` file stores our restaurant's menu items.
- The `week_1_sales` and `week_2_sales` files store our orders.

In [10]:
foods = pd.read_csv("restaurant_foods.csv")
customers = pd.read_csv("restaurant_customers.csv")
week1 = pd.read_csv("restaurant_week_1_sales.csv")
week2 = pd.read_csv("restaurant_week_2_sales.csv")

## The pd.concat Function I
- The `concat` function concatenates one **DataFrame** to the end of another.
- The original index labels will be kept by default. Set `ignore_index` to True to generate a new index.
- The `keys` parameter create a **MultiIndex** using the specified keys/labels.

In [11]:
week1.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [12]:
week2.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [13]:
len(week1), len(week2)

(250, 250)

In [26]:
pd.concat([week1, week2], keys=["week 1", "week 2"])

Customer ID  Food ID
week 1 0            537        9
       1             97        4
       2            658        1
       3            202        2
       4            155        9
...                 ...      ...
week 2 245          783       10
       246          556       10
       247          547        9
       248          252        9
       249          249        6

[500 rows x 2 columns]

In [35]:
df = pd.concat([week1, week2], keys=["week 1", "week 2"])
df.index.names

FrozenList([None, None])

In [38]:
# To assign names to your index levels:
df.index.names = ["week", "observation"]
# Now df.index.names returns: FrozenList(['week', 'observation'])
df.index.names

FrozenList(['week', 'observation'])

In [39]:
df = pd.concat([week1, week2], keys=["week 1", "week 2"], names=["week", "observation"])

In [24]:
pd.concat([week1, week2], ignore_index=True)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


## The pd.concat Function II
- Pandas will concatenate the **DataFrames** along the row/index axis.
- Pandas will include all columns that exist in either **DataFrame**. If there are no matching values, pandas will use `NaN` values.
- We can pass the `axis` parameter an argument of `"columns"` to concatenate on the column axis.

In [49]:
df1 = pd.DataFrame({"A": [1, 2, 3]})
df1

,A
0,1
1,2
2,3


In [51]:
df2 = pd.DataFrame({"B": [1, 2, 3]})
df2

,B
0,1
1,2
2,3


In [54]:
pd.concat([df1, df2], axis="columns")

,A,B
0,1,1
1,2,2
2,3,3


## Left Joins
- The `merge` method joins two **DataFrames** together based on shared values in a column or an index.
- A left join merges one **DataFrame** into another based on values in the first one.
- The "left" **DataFrame** is the one we invoke the `merge` method on.
- If the left **DataFrame's** value is not found in the right **DataFrame**, the row will hold `NaN` values.
<img src="SQL_Joins.png" width="800" height="800"/>

In [55]:
foods = pd.read_csv("restaurant_foods.csv")
customers = pd.read_csv("restaurant_customers.csv")
week1 = pd.read_csv("restaurant_week_1_sales.csv")
week2 = pd.read_csv("restaurant_week_2_sales.csv")

In [ ]:
week1

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,413,9
246,926,6
247,134,3
248,396,6


In [59]:
# Simplest - adds row in place
week1.loc[len(df)] = [413, 999]

In [60]:
week1

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
246,926,6
247,134,3
248,396,6
249,535,10


In [61]:
week1.head(), foods.head()

(   Customer ID  Food ID
 0          537        9
 1           97        4
 2          658        1
 3          202        2
 4          155        9,
    Food ID   Food Item  Price
 0        1       Sushi   3.99
 1        2     Burrito   9.99
 2        3        Taco   2.99
 3        4  Quesadilla   4.25
 4        5       Pizza   2.49)

In [62]:
week1.merge(foods, how="left", on="Food ID")

,Customer ID,Food ID,Food Item,Price
0,537,9,Donut,0.99
1,97,4,Quesadilla,4.25
2,658,1,Sushi,3.99
3,202,2,Burrito,9.99
4,155,9,Donut,0.99
...,...,...,...,...
246,926,6,Pasta,13.99
247,134,3,Taco,2.99
248,396,6,Pasta,13.99
249,535,10,Drink,1.75


## The left_on and right_on Parameters
- The `left_on` and `right_on` parameters designate the column names from each **DataFrame** to use in the merge.

In [64]:
week2.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [65]:
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [68]:
week2.merge(customers, how="left", left_on="Customer ID", right_on="ID").drop(
    "ID", axis="columns"
)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,Roger,Gordon,Male,Skilith,Operator
4,267,3,Matthew,Wood,Male,Agimba,Product Engineer
...,...,...,...,...,...,...,...
245,783,10,Phyllis,Meyer,Female,Voolia,Information Systems Manager
246,556,10,Samuel,Bailey,Male,Oyoloo,Nurse
247,547,9,Tina,Watkins,Female,Thoughtstorm,Accountant II
248,252,9,Douglas,Powell,Male,Jetwire,Geologist IV


## Inner Joins I
- Inner joins merge two tables based on *shared*/*common* values in columns.
- If only one **DataFrame** has a value, pandas will exclude it from the final results set.
- If the same ID occurs multiple times, pandas will store each possible combination of the values.
- The design of the join ensures that the results will be the same no matter what **DataFrame** the `merge` method is invoked upon.
<img src="SQL_Joins.png" width="800" height="800"/>

In [69]:
foods = pd.read_csv("restaurant_foods.csv")
customers = pd.read_csv("restaurant_customers.csv")
week1 = pd.read_csv("restaurant_week_1_sales.csv")
week2 = pd.read_csv("restaurant_week_2_sales.csv")

In [70]:
week1.merge(week2, how="inner")

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,922,1
6,21,4
7,578,5
8,578,5


## Inner Joins II
- We can pass multiple arguments to the `on` parameter of the `merge` method. Pandas will require matches in both columns across the **DataFrames**.

## Full/Outer Join
- A **full/outer** joins values that are found in either **DataFrame** or both **DataFrames**.
- Pandas does not mind if a value exists in one **DataFrame** but not the other.
- If a value does not exist in one **DataFrame**, it will have a `NaN`.

<img src="SQL_Joins.png" width="800" height="800"/>

## Merging by Indexes with the left_index and right_index Parameters
- Use the `on` parameter if the column(s) to be matched on have the same names in both **DataFrames**.
- Use the `left_on` and `right_on` parameters if the column(s) to be matched on have different names in the two **DataFrames**.
- Use the `left_index` or `right_index` parameters (set to True) to if the values to be matched on are found in the index of a **DataFrame**.

## The join Method
- The `join` method is a shortcut for concatenating two **DataFrames** when merging by index labels.